In [20]:
import subprocess
import findspark
import sys
import json 
import pandas as pd
import time 
import random
import os
import shutil
import requests
from datetime import datetime
from kafka import KafkaProducer
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [2]:
global dirGeneral
dirGeneral = "/opt/smart-parking/Python/GetData/Parkings/"

findspark.init(spark_home='/opt/spark')

topics = subprocess.check_output("/opt/kafka/bin/kafka-topics.sh --list --bootstrap-server hadoop-namenode:9092", shell=True)
topics = topics.split()
topics = [ topic.decode("UTF-8") for topic in topics ]
topics.pop()
topics

op = 0
while True:
    print("Para realizar el consumo del Parking seleccione\nalguno de los siguientes topicos de Kafka...")
    for i in range(1,len(topics)+1):
        print(str(i)+".- "+topics[i-1])
    try:
        o = int(input("Indique su opcion: "))
        if topics[o-1]:
            parking = topics[o-1]
        break
    except IndexError:
        print("seleccione una opcion correcta... ")

try:
    shutil.rmtree(dirGeneral+parking)
except FileNotFoundError as error:
    print("Directorio no existe, se creará directorio asociado al parking...")

os.makedirs(dirGeneral+parking+"/Areas/", exist_ok=False)

while True:
    op = input("Indique si realizo la carga del archivo config.json en el directorio correspondiente al parking (S o N): ")
    if op.lower() == "s":
        contenido = os.listdir(dirGeneral+parking)
        if "config.json" in contenido:
            break
        else:
            print("no se ha encontrado archivo config.json...")
    else:
        print("seleccione una opcion valida...")

if parking:
    print("\nEl parking seleccionado es - "+parking)  
    print("\n Cargando la configuracion necesaria para el consumo al backend")
    print("esta operacion puede tomar unos minutos...")
    
    with open(dirGeneral+parking+"/config.json", 'r') as f:
        baseConfigJson = json.load(f)
    
    #cargado de config para los levels...
    levels = [{"level_id": str(i+1)} for i in range(int(baseConfigJson['levels']))]
    with open(dirGeneral+parking+"/"+parking+"Levels.json", "w") as f:
        f.write(json.dumps({"levels": levels}, indent = 2))
        
    for i in range(len(baseConfigJson["areas"])):
        with open(dirGeneral+parking+"/Areas/areasLevel"+str(i+1)+".json", "w") as f:
            f.write(json.dumps({'areas':baseConfigJson["areas"][i]['areas_level_'+str(i+1)]}, indent = 4))
            
            
    # Spark session & context
    spark = (SparkSession
             .builder
             .master('local[*]')
             .appName(parking+"Spark")
             .config('spark.jars', 'file:///opt/smart-parking/Python/Spark-Jars-Utils/spark-sql-kafka-0-10_2.12-3.2.1.jar,file:///opt/smart-parking/Python/Spark-Jars-Utils/kafka-clients-3.1.0.jar')
             .config('spark.executor.extraClassPath','file:///opt/smart-parking/Python/Spark-Jars-Utils/spark-sql-kafka-0-10_2.12-3.2.1.jar:file:///opt/smart-parking/Python/Spark-Jars-Utils/kafka-clients-3.1.0.jar')
             .config('spark.executor.extraLibrary','file:///opt/smart-parking/Python/Spark-Jars-Utils/spark-sql-kafka-0-10_2.12-3.2.1.jar:file:///opt/smart-parking/Python/Spark-Jars-Utils/kafka-clients-3.1.0.jar')
             .config('spark.driver.extraClassPath', 'file:///opt/smart-parking/Python/Spark-Jars-Utils/spark-sql-kafka-0-10_2.12-3.2.1.jar:file:///opt/smart-parking/Python/Spark-Jars-Utils/kafka-clients-3.1.0.jar')
             .getOrCreate())

    sc = spark.sparkContext
    sc.setLogLevel("ERROR")


    # Subscribe to 1 topic
    df = spark \
      .readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "hadoop-namenode:9092") \
      .option("subscribe", parking) \
      .load()
    df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
else:
    sys.exit()

Para realizar el consumo del Parking seleccione
alguno de los siguientes topicos de Kafka...
1.- PuntaCarretasShopping
2.- TresCrucesShopping
3.- XBEESmartParkingModel
Indique su opcion: 2
Indique si realizo la carga del archivo config.json en el directorio correspondiente al parking (S o N): 2
seleccione una opcion valida...
Indique si realizo la carga del archivo config.json en el directorio correspondiente al parking (S o N): s
no se ha encontrado archivo config.json...
Indique si realizo la carga del archivo config.json en el directorio correspondiente al parking (S o N): s

El parking seleccionado es - TresCrucesShopping

 Cargando la configuracion necesaria para el consumo al backend
esta operacion puede tomar unos minutos...


22/06/05 22:31:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [36]:
def percentagetoString(sp_total, sp_ocupied):
    percnt = int((sp_ocupied*100)/sp_total)
    #print(percnt)
    if percnt >= 0 and percnt <=25:
        return "Empty"
    elif percnt > 25 and percnt <=50:
        return "Almost Empty"
    elif percnt > 50 and percnt <=75:
        return "Almost Full"
    elif percnt > 75 and percnt <=100:
        return "Full"
    else:
        return "Other"

def process_row(row):
    producer = KafkaProducer(bootstrap_servers=['hadoop-namenode:9092'], api_version=(0,10,0))
    
    row = json.loads(row['value'].decode("UTF-8"))
    
    #Routine for Slots
    totalSpots = row["device_spots"]
    row['slotsUpdated'] = []
    occu = 0
    for i in range(len(row["device_slots"])):
        if row["device_slots"][i] == True:
            occu += 1
        row['slotsUpdated'].append((str(i+1), row["device_slots"][i]))
    row['area_occupation'] = percentagetoString(int(totalSpots), occu)
    
    row['area_available_spots'] = str(int(totalSpots) - occu)
    row['area_occupied_spots'] = str(occu)
    row['area_total_spots'] = totalSpots
    areaInfo = {}
    areaInfo.update(area_id=row['device_area_id'],area_name=row['device_area_name'],area_description="shot description",
                    area_occupation=row['area_occupation'], area_occupied_spots=row['area_occupied_spots'],
                    area_total_spots=row['area_total_spots'], area_available_spots=row['area_available_spots']) 
    
    #TO DO
    areaInfo['area_average_price'] = "200"
    
    slotsInfo = []
    for slot in row['slotsUpdated']:
        SLTINFO = {}
        SLTINFO['slot_id'] = slot[0]
        SLTINFO['slot_state'] = slot[1]
        #TO DO
        SLTINFO['slot_price'] = "200"
        slotsInfo.append(SLTINFO)
    
    areaInfo.update(slots=slotsInfo)
   
    # AREAS DATA
    with open("/opt/smart-parking/Python/GetData/Parkings/"+row["parking_name"]+'/Areas/areasLevel'+row['device_level_id']+'.json', 'r') as f:
        JsonAreas = json.load(f)  

    for i in range(len(JsonAreas['areas'])):
        if str(areaInfo['area_id']) == JsonAreas['areas'][i]['area_id'] and str(areaInfo['area_name']) == JsonAreas['areas'][i]['area_name']:
            JsonAreas['areas'][i] = areaInfo
                
    with open("/opt/smart-parking/Python/GetData/Parkings/"+row["parking_name"]+'/Areas/areasLevel'+row['device_level_id']+'.json', 'w') as f:
        f.write(json.dumps(JsonAreas, indent = 2))
    
    ## LEVELS DATA
    levelInfo = {}
    levelInfo.update(level_id=row['device_level_id'], level_name="Piso "+row['device_level_id']) 
    
    # TO DO
    levelInfo['level_average_price'] = "200"

    sumaOccuLevelSlots = 0
    sumaAvailableLevelSlots = 0
    sumaTotalLevelSlots = 0
    for area in JsonAreas['areas']:
        if "area_occupied_spots" in area:
            sumaOccuLevelSlots += int(area['area_occupied_spots'])
            sumaTotalLevelSlots += int(area['area_total_spots'])
            sumaAvailableLevelSlots += int(area['area_available_spots'])
        
    
    levelInfo.update(level_occupation=percentagetoString(sumaTotalLevelSlots, sumaOccuLevelSlots), level_occupied_spots=sumaOccuLevelSlots,
                    level_available_spots= str(sumaAvailableLevelSlots),level_total_spots=str(sumaTotalLevelSlots), areas=JsonAreas['areas'])
    
    #levelInfo.update(level_occupation=percentagetoString(sumaTotalLevelSlots, sumaOccuLevelSlots), level_occupied_slots=sumaOccuLevelSlots,
    #                level_total_slots=sumaTotalLevelSlots, areas=JsonAreas['areas'])
    
    with open("/opt/smart-parking/Python/GetData/Parkings/"+row["parking_name"]+'/'+row["parking_name"]+'Levels.json', 'r') as f:
        JsonLevels = json.load(f)  
    #print(JsonLevels)
    for i in range(len(JsonLevels['levels'])):
        if str(levelInfo['level_id']) == JsonLevels['levels'][i]['level_id']:
            JsonLevels['levels'][i] = levelInfo
                
    with open("/opt/smart-parking/Python/GetData/Parkings/"+row["parking_name"]+'/'+row["parking_name"]+'Levels.json', 'w') as f:
        f.write(json.dumps(JsonLevels, indent = 2))
    
    parkingInfo = {"parking_id": row["parking_id"]}
    parkingInfo.update(parking_name=row['parking_name'], parking_description=row['parking_description'],
                      parking_address=row['parking_address'], parking_closed=False, parking_latitude=str(row['parking_latitude']),
                      parking_longitude=str(row['parking_longitude']), levels=JsonLevels['levels'])
    parkingInfo['parking_timestamp'] = datetime.now().strftime("%d/%m/%Y, %H:%M:%S")
    
    # print(datetime.strptime(datetime.now(), "%Y-%m-%d %H:%M"))
    #producer.send("Parkings", value=json.dumps(parkingInfo).encode('utf-8'))
    #producer.close()
    
    r = requests.post('https://smart-parking-ort-db.herokuapp.com/api/v1/parkings/'+row["parking_id"], json=parkingInfo)
    print(f"Status Code: {r.status_code}, Response: {r.json()}")
    
    print(parkingInfo['parking_timestamp'])
    #print(json.dumps(parkingInfo, indent = 2))
    
    #time.sleep(1)
    

In [ ]:
df \
.writeStream\
.outputMode("append") \
.foreach(process_row) \
.start()\
.awaitTermination()

Status Code: 200, Response: {'message': 'updated'}                  (0 + 1) / 1]
06/06/2022, 12:53:14
Status Code: 200, Response: {'message': 'updated'}                  (0 + 1) / 1]
06/06/2022, 12:53:15
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:53:15
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:53:16
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:53:17
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:53:17
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:53:18
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:53:19
Status Code: 200, Response: {'message': 'updated'}                  (0 + 1) / 1]
06/06/2022, 12:53:20
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:53:20
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:53:21
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:53:22
Status Code: 200, Response: {'message': 'updat

Status Code: 200, Response: {'message': 'updated'}                  (0 + 1) / 1]
06/06/2022, 12:54:38
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:54:39
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:54:40
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:54:40
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:54:41
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:54:42
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:54:43
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:54:43
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:54:44
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:54:45
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:54:45
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:54:46
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:54:47
Status Code: 200, Response: {'mess

Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:56:50
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:56:51
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:56:51
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:56:52
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:56:52
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:56:53
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:56:54
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:56:54
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:56:55
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:56:56
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:56:56
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:56:57
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:56:58
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 1

Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:58:14
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:58:15
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:58:16
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:58:16
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:58:17
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:58:18
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:58:19
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:58:19
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:58:20
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:58:21
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:58:21
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:58:22
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:58:23
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 1

Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:59:35
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:59:35
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:59:36
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:59:37
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:59:37
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:59:38
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:59:39
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:59:39
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:59:40
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:59:41
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:59:41
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:59:42
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 12:59:43
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 1

...

Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:47:57
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:47:58
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:47:58
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:47:59
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:48:00
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:48:00
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:48:01
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:48:02
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:48:03
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:48:03
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:48:04
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:48:05
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:48:05
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 1

Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:49:19
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:49:20
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:49:21
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:49:36
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:49:37
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:49:37
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:49:38
Status Code: 200, Response: {'message': 'updated'}                  (0 + 1) / 1]
06/06/2022, 13:49:39
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:49:39
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:49:40
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:49:41
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:49:41
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:49:42
Status Code: 200, Response: {'mess

Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:50:56
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:50:57
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:50:58
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:50:59
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:50:59
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:51:00
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:51:01
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:51:01
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:51:02
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:51:03
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:51:03
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:51:04
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:51:05
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 1

Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:52:20
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:52:21
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:52:21
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:52:23
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:52:24
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:52:25
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:52:25
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:52:26
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:52:27
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:52:27
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:52:28
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:52:29
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:52:29
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 1

Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:53:44
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:53:44
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:53:45
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:53:46
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:53:46
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:53:47
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:53:48
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:53:49
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:53:49
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:53:50
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:53:51
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:53:51
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 13:53:52
Status Code: 200, Response: {'message': 'updated'}
06/06/2022, 1